In [1]:
import pandas as pd
import plotly.express as px
import dash
import dash_core_components as dcc
import dash_html_components as html
from dash.dependencies import Input, Output

# spreadsheet directory
workbook = './data/ga_rawdata.xlsx'

# read all excel sheets
all_dfs = pd.read_excel(workbook, sheet_name=None)

# drop unwanted sheet from dictionary
removed = all_dfs.pop('Copy of Report Configuration')

merged_df = pd.DataFrame()

# for iteration to modify all dataframes
for df in all_dfs:

    # drop possibly empty columns 
    all_dfs[df] = all_dfs[df].iloc[:,0:5]

    # rename columns (drop 'ga:')
    for col in all_dfs[df].columns:
        all_dfs[df].rename(columns={col : col[3:]}, inplace=True)
    
    # create month and year columns for easier manipulation & drop yearMonth   
    all_dfs[df]['year'] = all_dfs[df]['yearMonth'].apply(lambda x: str(x)[:4])
    all_dfs[df]['month'] = all_dfs[df]['yearMonth'].apply(lambda x: str(x)[4:])
    all_dfs[df].drop(columns=['yearMonth'], inplace=True)
    

    # merge all dataframes 
    all_dfs[df]['market'] = df

    merged_df = pd.concat(all_dfs, ignore_index=True)

# create summary dataframe
summary_df = merged_df.groupby(['market','year']).agg({
                    'users':'mean',
                    'bounces' : 'mean',
                    'pageviews' : 'mean',
                    'sessions' : 'mean',
                })

# rename columns as average
for col in summary_df.columns:
    summary_df.rename(columns={col:str('average_'+col)}, inplace=True)

# sort values based on market number
summary_df.reset_index(inplace=True)
summary_df['labels'] = summary_df.market.str[-2:]
summary_df = summary_df.sort_values('labels').drop('labels', axis=1)
summary_df.dropna(inplace=True)

# create a datetime column
merged_df.dropna(inplace=True)
merged_df['date'] = pd.to_datetime(merged_df[['year','month']].assign(DAY=1))

In [2]:
merged_df

,users,sessions,pageviews,bounces,year,month,market,date
0,0.0,0.0,0.0,0.0,2009,01,Market 1,2009-01-01
1,0.0,0.0,0.0,0.0,2009,02,Market 1,2009-02-01
2,26842.0,33513.0,181997.0,18286.0,2009,03,Market 1,2009-03-01
3,35800.0,55654.0,498481.0,22716.0,2009,04,Market 1,2009-04-01
4,43006.0,68553.0,548333.0,29195.0,2009,05,Market 1,2009-05-01
...,...,...,...,...,...,...,...,...
899,6269127.0,12068971.0,37803533.0,5886893.0,2014,09,Market 12,2014-09-01
900,5850036.0,10979049.0,32880260.0,5579772.0,2014,10,Market 12,2014-10-01
901,5917161.0,11465399.0,34547591.0,5805290.0,2014,11,Market 12,2014-11-01
902,3665866.0,6550869.0,16030402.0,3843473.0,2014,12,Market 12,2014-12-01
